In [1]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.svm import SVC

In [2]:

# Load the datasets
train_df = pd.read_csv("train_users.csv")
test_df = pd.read_csv("test_users.csv")
edu_df = pd.read_csv("education.csv")
skills_df = pd.read_csv("skills.csv")
lang_df = pd.read_csv("languages.csv")
workexp_df = pd.read_csv("work_experiences.csv")
df_subm = pd.read_csv("submission.csv")


In [3]:
workexp_df

,user_id,company_id,location,start_year_month
0,53442,2651,"Istanbul, Turkey",201509
1,34558,815,"Istanbul, Turkey",201210
2,63761,26354,NaN,200010
3,10738,89,NaN,201610
4,8711,3113,"Istanbul, Turkey",201801
...,...,...,...,...
187014,22180,15065,Gebze,201205
187015,55822,25076,stajer,201506
187016,13750,1607,NaN,201901
187017,3679,1414,"Ankara, Turkey",201807


In [4]:
#sns.heatmap(workexp_df.corr(),cmap= "YlGnBu")
#plt.show()
target = 'moved_after_2019'
user_id = 'user_id'


In [5]:
train_df = train_df.set_index(user_id)
test_df = test_df.set_index(user_id)
df_subm = df_subm.set_index(user_id)

workexp_df = workexp_df[workexp_df['start_year_month'] < 201901] # see https://www.kaggle.com/competitions/garanti-bbva-data-camp/discussion/383774

In [6]:

# Merge the datasets
train = pd.merge(train_df, edu_df, on='user_id', how='left')
train = pd.merge(train, skills_df, on='user_id', how='left')
train = pd.merge(train, lang_df, on='user_id', how='left')
train = pd.merge(train, workexp_df, on='user_id', how='left')

test = pd.merge(test_df, edu_df, on='user_id', how='left')
test = pd.merge(test, skills_df, on='user_id', how='left')
test = pd.merge(test, lang_df, on='user_id', how='left')
test = pd.merge(test, workexp_df, on='user_id', how='left')

In [7]:
edu_df = edu_df.applymap(lambda x: x.lower() if type(x) == str else x)
train_df = train_df.applymap(lambda x: x.lower() if type(x) == str else x)
test_df = test_df.applymap(lambda x: x.lower() if type(x) == str else x)
skills_df = skills_df.applymap(lambda x: x.lower() if type(x) == str else x)
lang_df = lang_df.applymap(lambda x: x.lower() if type(x) == str else x)
workexp_df = workexp_df.applymap(lambda x: x.lower() if type(x) == str else x)

In [8]:
edu_df['degree'] = edu_df['degree'].str.lower()


In [9]:
missing_values_count = edu_df.isnull().sum()

print(missing_values_count)

user_id                  0
school_name              0
degree               35842
fields_of_study      19838
start_year_month    134373
end_year_month      134985
dtype: int64


In [10]:

# Define the list of terms to check for in the school_name column
list_bachelor = ['üniversite', 'university', 'uni', 'üni', 'fakülte', 'faculty', 'enstitü', 'enstitude']


# Create a boolean mask to filter rows with the terms in my_list in the school_name column
mask = edu_df['school_name'].str.contains('|'.join(list_bachelor))

# Set the value of the degree column to 'bachelor' for the filtered rows with a blank degree
edu_df.loc[mask & edu_df['degree'].isnull(), 'degree'] = 'bachelor'

In [11]:
edu_df.fillna({'degree':'0'}, inplace=True)

In [12]:
#bu değerleri liseye eşitle 
mask = edu_df['degree'].str.contains('high school', 'lise')

# replace the values in the selected rows with 'lise'
edu_df.loc[mask, 'degree'] = 'lise'
mask = edu_df['degree'].str.contains("bachelor's")

edu_df.loc[mask, 'degree'] = 'bachelor'

mask = edu_df['degree'].str.contains('bache', 'bachelor')

edu_df.loc[mask, 'degree'] = 'bachelor'
mask = edu_df['degree'].str.contains('school')

edu_df.loc[mask, 'degree'] = 'lise'
mask = edu_df['degree'].str.contains('lise','li̇se')

edu_df.loc[mask, 'degree'] = 'lise'

mask = edu_df['degree'].str.contains('phd')

edu_df.loc[mask, 'degree'] = 'phd'

mask = edu_df['degree'].str.contains("master's")

edu_df.loc[mask, 'degree'] = 'master'
mask = edu_df['degree'].str.contains('doktora')

edu_df.loc[mask, 'degree'] = 'phd'
mask = edu_df['degree'].str.contains('ph.d.', 'doktora')

edu_df.loc[mask, 'degree'] = 'phd'
mask = edu_df['degree'].str.contains('doctor', 'dr.')

edu_df.loc[mask, 'degree'] = 'phd'
mask = edu_df['degree'].str.contains('master')

edu_df.loc[mask, 'degree'] = 'master'
mask = edu_df['degree'].str.contains('orta', 'ilk')

edu_df.loc[mask, 'degree'] = 'orta'

mask = edu_df['degree'].str.contains('i̇lk', 'ilk')

edu_df.loc[mask, 'degree'] = 'orta'


In [13]:
keywords = ['ba', 'b. s.', 'b. sc.', 'degre', 'forma', 'associate', 'li̇sans', 'sert', 'tech', 'web',
                'scien', 'certif', 'bachelorlorlor', 'myo', 'hon', 'bi̇li̇şi̇m',
                'bilşim' 'bilişim', 'bir', 'program', 'bache', 'bi̇li̇m', 'bilim',
                'lisa', 'faculty', 'licence degree', 'mezunu.', 'mühendisligi',
                'sınıf', '1.', '2.', '3.', '4.', 'hazırlık', 'hazirlik', 'fakültesi',
                'fakulte', 'fakülte', 'faculty', 'lisans', 'university', 'üniversity',
                'üniversite', 'licence', 'bs', 'b.s', 'b.sc', 'engineer', 'mühendis',
                'muhendis', 'mezun', 'graduate', 'licentiate', 'bacheloru', 'bachelornce',
                'fakulty', 'ĺisans', 'bachelorter' 'meslek yuksek', 'bilgisayar', 'yuksek',
                'bachelorsi', 'licance', 'm.y.o', 'license', 'bachelorns', 'bachelorsinif',
                'erasmus', 'yazılım', 'öğrenci', 'comp.', 'eng', 'computer', 'bolum', 'bölüm',
                'bölümü', 'bolumu', 'bilgi', 'advanced', 'kurs', 'academy', 'nanodegree', 'belge', 'executive education']

edu_df.loc[:, 'degree'] = edu_df.loc[:, 'degree'].apply(lambda x: 'bachelor' if any(word in x for word in keywords) else x)


In [14]:
replace_list_2 = ['--', 'lise','li̇se','meslek', 'lıse', 'course', 'student', 'exchange lise']

# Replace the values in the 'degree' column that contain the strings in the replace_list
edu_df['degree'] = edu_df['degree'].str.lower().replace('|'.join(replace_list_2), 'lise', regex=True)


In [15]:
replace_list_3 = ['masterc', 'master.', 'yüksek', 'master','mba', 'm.s.', 'M.Sc', 'MS', 'ma', 'm.a.',
                  'MBA', 'M.S', 'master of', 'm.sc.', 'ms', 'm.s', 'm.sc', 'bcomaster']

# Replace the values in the 'degree' column that contain the strings in the replace_list
edu_df['degree'] = edu_df['degree'].str.lower().replace('|'.join(replace_list_3), 'master', regex=True)


In [16]:
replace_list_4 = ['doktora', 'doktora (dr.)', 'philosophy', 'ph.d.' , 'doctor', 
                  'phd', 'ph.d', 'phd of', 'doctor of philosophy (ph.d.)']

# Replace the values in the 'degree' column that contain the strings in the replace_list
edu_df['degree'] = edu_df['degree'].str.lower().replace('|'.join(replace_list_2), 'phd', regex=True)

In [18]:
 
num_unique = edu_df['degree'].nunique()

# print the number of unique values
print(f"There are {num_unique} unique values in the 'degree' column.")

unique_values_degree = edu_df['degree'].unique()

for value in unique_values_degree:
    print(value)

    

There are 857 unique values in the 'degree' column.
bachelor
phd
master
teknisyen
diplomaster
 
bilişim teknolojileri
teaching
şuan okuyorum
mastertemastertik - fen
orta
instructor
dok
y
درجة البكالوريوس 
ielts/toefl
turkce language
-windows formasterplications, masterl, xml
ilköğretim
okul dördüncüsü
elektronik teknikeri
elektronik teknisyeni
yok 
bilişim teknolojileri 
terk
ph. d
masterthemastertics
tekniker
systemasterlution expert
occupational safety specialist (i̇ş güvenliği uzmasternı)
post.doc.
introduction to cybersecurity
mobility fundamentals
department of economics
vocational college
i̇şletme
phd master okulu
çift anadal/double masterjor
-
uzaktan eğitim
preparation
dropped out
mmf
i̇ngilizce
it institute
suspended
phd masterokulu
masterokul
master onur
new media
grafik tasarım
özel eğitim
advance
masterc
burslu
sosyoloji
visual communication design
very good
london
elektronik ve haberleşme müh.
college
health officer
distance education
minor
yandal
fourth grade phd 
office 

In [17]:
#numeric ve floatları en çok yinelenen değer olan bachelor ile değiştir
edu_df['degree'] = edu_df['degree'].apply(lambda x: 'bachelor' if bool(re.search(r'\d', str(x))) else x)


In [ ]:
df_exp.loc[df_exp['location'].str.contains("ankara|odtü|metu|hacettepe|bilkent", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "ankara"
df_exp.loc[df_exp['location'].str.contains("izmir|i̇zmir,|i̇zmir|i̇zmi̇r", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "izmir"
df_exp.loc[df_exp['location'].str.contains("gayrettepe|mecidiyekoy|ayazağa|yıldız|küçükyalı|maltepe|itu|itü|taksim|levent|çerkezköy|kozyatağı|marmara|kadıköy|istanbul|pendik|tuzla|çekmeköy|i̇stanbul|istanbul,|istanb|üsküdar|beyoğlu|yıldız teknik|ytü|kartal|maslak|ümraniye|beşiktaş|besiktas|ataşehir", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "istanbul"
df_exp.loc[df_exp['location'].str.contains("adana", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "adana"
df_exp.loc[df_exp['location'].str.contains("kocaeli|izmit|gebze|gölcük", na=False) & ~df_exp['location'].str.contains("other_string"), 'location'] = "kocaeli"


In [32]:
degree_counts = edu_df['school_name'].value_counts().head(30)
print(degree_counts)

sakarya üniversitesi                      3364
i̇stanbul teknik üniversitesi             3291
istanbul technical university             3291
i̇stanbul üniversitesi                    3232
kocaeli üniversitesi                      3154
yıldız teknik üniversitesi                3139
marmara üniversitesi                      2749
hacettepe üniversitesi                    2638
anadolu üniversitesi                      2545
ege üniversitesi                          2341
gazi üniversitesi                         2265
bahçeşehir üniversitesi                   2016
dokuz eylül üniversitesi                  1833
yildiz technical university               1790
istanbul university                       1647
beykent üniversitesi                      1501
boğaziçi üniversitesi                     1386
ankara üniversitesi                       1381
karadeniz teknik üniversitesi             1336
eskişehir osmangazi üniversitesi          1315
trakya üniversitesi                       1303
selçuk üniver

In [20]:
 
num_unique = edu_df['degree'].nunique()

# print the number of unique values
print(f"There are {num_unique} unique values in the 'degree' column.")

unique_values_degree = edu_df['degree'].unique()

for value in unique_values_degree:
    print(value)


There are 857 unique values in the 'degree' column.
bachelor
phd
master
teknisyen
diplomaster
 
bilişim teknolojileri
teaching
şuan okuyorum
mastertemastertik - fen
orta
instructor
dok
y
درجة البكالوريوس 
ielts/toefl
turkce language
-windows formasterplications, masterl, xml
ilköğretim
okul dördüncüsü
elektronik teknikeri
elektronik teknisyeni
yok 
bilişim teknolojileri 
terk
ph. d
masterthemastertics
tekniker
systemasterlution expert
occupational safety specialist (i̇ş güvenliği uzmasternı)
post.doc.
introduction to cybersecurity
mobility fundamentals
department of economics
vocational college
i̇şletme
phd master okulu
çift anadal/double masterjor
-
uzaktan eğitim
preparation
dropped out
mmf
i̇ngilizce
it institute
suspended
phd masterokulu
masterokul
master onur
new media
grafik tasarım
özel eğitim
advance
masterc
burslu
sosyoloji
visual communication design
very good
london
elektronik ve haberleşme müh.
college
health officer
distance education
minor
yandal
fourth grade phd 
office 

In [21]:
missing_values_count = edu_df.isnull().sum()

print(missing_values_count)


user_id                  0
school_name              0
degree                   0
fields_of_study      19838
start_year_month    134373
end_year_month      134985
dtype: int64


In [22]:
edu_df.fillna({'fields_of_study':'0'}, inplace=True)

In [23]:
edu_df.fillna({'start_year_month':'0'}, inplace=True)

In [24]:
edu_df.fillna({'end_year_month':'0'}, inplace=True)

In [25]:
edu_df.fillna({'skill':'0'}, inplace=True)

In [26]:
edu_df.fillna({'language':'0'}, inplace=True)


In [27]:
edu_df.fillna({'proficiency':'0'}, inplace=True )


In [28]:
edu_df = edu_df.drop_duplicates(['user_id', 'degree'])


In [29]:
def get_degree_category(degree):
    if degree.lower().startswith('graduate'):
        return 'graduate'
    elif degree.lower().startswith('phd'):
        return 'phd'
    elif degree.lower().startswith('master'):
        return 'master'
    elif degree.lower().startswith('other'):
        return 'other'
    

edu_df['new_degree_category'] = edu_df['degree'].apply(get_degree_category)

new_edu = edu_df.groupby('user_id')['new_degree_category'].value_counts().unstack(fill_value=0).reset_index()
new_edu.columns.name = None
new_edu.columns = ['user_id', 'graduate','phd', 'master' , 'other']

ValueError: Length mismatch: Expected axis has 4 elements, new values have 5 elements

In [ ]:
 

    
degree_counts = lang_df['language'].value_counts().head(30)
print(degree_counts)

In [ ]:
lang_rank = {lang: rank for rank, lang in enumerate(['elementary', 'limited_working', 'professional_working', 'full_professional', 'native_or_bilingual'], 1)}

lang_df['proficiency'] = lang_df['proficiency'].map(lang_rank)


In [ ]:
import re

lang_conv = {
    'english': ['ingilizce', 'english', 'englishch', 'englisch', 'i̇ngilizce'],
    'german': ['almanca', 'german', 'deutsch'],
    'french': ['fransızca', 'french', 'français'],
    'spanish': ['ispanyolca', 'spanish', 'español'],
    'arabic': ['arapça', 'arabic'],
    'russian': ['rusça', 'russian'],
    'japanese': ['japanese(beginner)', 'roomaji', 'japonca', ],
    'italian': ['italian', 'i̇talyanca', 'italiano', ],
    'turkish': ['türkçe', 'turkish', 'türkisch']
    
}

lang_df['language'] = lang_df['language'].apply(lambda x: next((k for k, v in lang_conv.items() if re.match(f'(?i)({"|".join(v)})', x)), x))
lang_df = lang_df[lang_df['language'].isin(lang_conv.keys())]


In [ ]:
lang_df = lang_df.drop_duplicates(['user_id', 'language'])
lang_df = pd.pivot(lang_df, index='user_id', columns='language', values='proficiency')
lang_df = lang_df.fillna(0).astype(int)
lang_df.head()

In [ ]:
skills_df.columns

In [ ]:
used_skills = skills_df['skill'].value_counts().iloc[:90].index # we only took most common 60 skills, you can increase it

skills_df = skills_df[skills_df['skill'].isin(used_skills)]
skills_df['experience'] = True

In [ ]:
skills_df = skills_df.drop_duplicates(['user_id', 'skill'])
skills_df = pd.pivot(skills_df, index='user_id', columns='skill', values='experience')
skills_df = skills_df.fillna(0).astype(int)
skills_df.head()

In [ ]:
workexp_df = workexp_df.sort_values(by=['user_id', 'start_year_month']) # we want nth function to return the last companies by date
df_new = pd.DataFrame()

In [ ]:
df_new['company(1th)'] = workexp_df.groupby(user_id)['company_id'].nth(-1).astype(str)
df_new['company(2th)'] = workexp_df.groupby(user_id)['company_id'].nth(-2).astype(str)
df_new['company(3th)'] = workexp_df.groupby(user_id)['company_id'].nth(-3).astype(str)

df_new['company_location(1th)'] = workexp_df.groupby(user_id)['location'].nth(-1).astype(str)
df_new['company_location(2th)'] = workexp_df.groupby(user_id)['location'].nth(-2).astype(str)
df_new['company_location(3th)'] = workexp_df.groupby(user_id)['location'].nth(-3).astype(str)

df_new['min_exp_time'] = workexp_df.groupby(user_id)['start_year_month'].min()
df_new['max_exp_time'] = workexp_df.groupby(user_id)['start_year_month'].max()

df_new['company_count_2018'] = workexp_df[workexp_df['start_year_month'].gt(201712)].groupby(user_id).size()
df_new['company_count_2017'] = workexp_df[workexp_df['start_year_month'].gt(201612)].groupby(user_id).size()
df_new['company_count_2016'] = workexp_df[workexp_df['start_year_month'].gt(201512)].groupby(user_id).size()

In [ ]:
workexp_df = df_new
workexp_df.head()


In [ ]:
test_df

In [ ]:
train_df

In [ ]:
skills_df

In [30]:
train_df[edu_df.columns]   = edu_df[edu_df.columns]
train_df[lang_df.columns]  = lang_df[lang_df.columns]
train_df[skills_df.columns] = skills_df[skills_df.columns]
train_df[workexp_df.columns] = workexp_df[workexp_df.columns]

test_df[edu_df.columns]   = edu_df[edu_df.columns]
test_df[lang_df.columns]  = lang_df[lang_df.columns]
test_df[skills_df.columns] = skills_df[skills_df.columns]
test_df[workexp_df.columns] = workexp_df[workexp_df.columns]


In [31]:
test_df

,industry,location,user_id,school_name,degree,fields_of_study,start_year_month,end_year_month,new_degree_category,language,proficiency,skill,company_id
user_id,,,,,,,,,,,,,
17449,research,sales consultant,38048.0,yildiz technical university,bachelor,computer science & engineering,199605.0,0,None,i̇ngilizce,NaN,google analytics,1607.0
33967,computer software,"duzce, turkey",42411.0,NaN,NaN,NaN,201008.0,NaN,None,german,professional_working,ekip çalışması,740.0
2110,automotive,"ankara, türkiye",40351.0,fırat üniversitesi,bachelor,bilgisayar yazılımı mühendisliği,201110.0,0,None,türkçe,native_or_bilingual,database design,7674.0
55082,internet,NaN,7132.0,yeditepe üniversitesi,bachelor,computer science and engineering,201806.0,0,None,türkçe,native_or_bilingual,c#,944.0
37165,electrical/electronic manufacturing,"istanbul, turkey",10677.0,sakarya university,bachelor,electrical and electronics engineering,201703.0,202105.0,None,almanca,elementary,microsoft sql server,1724.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32847,computer software,"istanbul, turkey",17631.0,dokuz eylul university,bachelor,ekonomi,201609.0,0,None,turkish,NaN,ccna,6169.0
20054,hospital & health care,"istanbul, turkey",33682.0,catholic university institute of buea,bachelor,software engineering,200507.0,0,None,german,native_or_bilingual,project lombok,18968.0
7029,wireless,"istanbul, turkey",20425.0,karadeniz teknik üniversitesi,bachelor,computer software engineering,201607.0,0,None,german,elementary,yazılım dokümantasyonu,3488.0


In [ ]:
cat_cols = [col for col in test_df.columns if test_df[col].dtype == 'object']
num_cols = [col for col in test_df.columns if test_df[col].dtype != 'object']

In [ ]:

for col in cat_cols:
    train_cats  = set(train_df[col].unique())
    test_cats   = set(test_df[col].unique())
    common_cats = set.intersection(train_cats, test_cats)
    
    train_df.loc[~train_df[col].isin(common_cats), col] = 'other'
    test_df.loc[~test_df[col].isin(common_cats), col] = 'other'

In [ ]:
df_all = pd.concat([train_df, test_df], axis=0)

for col in cat_cols:
    df_all[col] = df_all[col].factorize()[0]

df_all[cat_cols] = df_all[cat_cols].astype('category')
df_all[num_cols] = df_all[num_cols].fillna(0)

train_df = df_all.loc[train_df.index, train_df.columns]
test_df = df_all.loc[test_df.index, test_df.columns]

In [ ]:
train_df = train_df.dropna()


In [ ]:
train_df

In [ ]:
train_df

In [ ]:
X, y = train_df.drop(columns=[target]), train_df[target]
clf = RandomForestClassifier(n_estimators=300, class_weight='balanced_subsample', random_state=42)
cv  = StratifiedKFold(shuffle=True, random_state=42)

scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')

In [ ]:
print(f'Scores mean: {np.mean(scores):.4f}')
print(f'Scores std: {np.std(scores):.4f}')

In [ ]:
clf.fit(X, y)
df_subm.loc[test_df.index, target] = clf.predict(test_df)

df_subm[target] = df_subm[target].astype(int)
df_subm[target].value_counts()


In [ ]:
#df_subm[target] = df_subm[target].clip(lower=0)  # Set negative values to 0
#df_subm[target] = df_subm[target].astype(int)  # Convert to integers
#df_subm.to_csv('submission.csv', index=False)  # Save the submission file


In [ ]:
df_subm.to_csv('submission.csv')


In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier


# Split the data into features (X) and target (y)
X = train_df.drop(columns=[target], axis=1)
y = train_df[target]

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a KNN classifier with k=5 and fit the model to the training data
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Make predictions on the test data
y_pred = knn.predict(X_test)

# Evaluate the accuracy of the model on the test data
score = knn.score(X_test, y_test)
print('KNN model score is %0.4f' % score)



In [ ]:
df_subm.loc[test_df.index, target] = knn.predict(test_df)

df_subm[target] = df_subm[target].astype(int)
df_subm[target].value_counts()


In [ ]:
df_subm.to_csv('submission.csv')


In [ ]:
test_df

In [ ]:
test_df. columns. values

In [ ]:
le = LabelEncoder()
train_df['moved_after_2019'] = le.fit_transform(train_df['moved_after_2019'])
features = ['industry', 'location', 'user_id', 'school_name', 'degree',
       'fields_of_study', 'start_year_month', 'end_year_month',
       'new_degree_category', 'arabic', 'english', 'french', 'german',
       'russian', 'spanish', 'turkish', '.net', '.net core',
       'agile methodologies', 'agile metotları', 'ajax', 'android',
       'android development', 'angular', 'angularjs', 'araştırma',
       'asp.net', 'asp.net mvc', 'autocad', 'bootstrap', 'c', 'c#', 'c++',
       'cascading style sheets (css)', 'css', 'design patterns', 'docker',
       'eclipse', 'ekip çalışması', 'entity framework', 'git',
       'hibernate', 'html', 'html5', 'i̇ngilizce', 'java', 'javascript',
       'jira', 'jquery', 'json', 'liderlik', 'linq', 'linux',
       'machine learning', 'management', 'matlab', 'maven',
       'microsoft excel', 'microsoft office', 'microsoft sql server',
       'microsoft word', 'mongodb', 'mysql', 'mühendislik', 'node.js',
       'object oriented design', 'object-oriented programming (oop)',
       'oop', 'oop (nesne yönelimli programlama)', 'oracle', 'photoshop',
       'php', 'pl/sql', 'postgresql', 'powerpoint', 'programlama',
       'programming', 'proje yönetimi', 'project management', 'python',
       'python (programming language)', 'react.js', 'scrum', 'soap',
       'software design', 'software development', 'software engineering',
       'software project management', 'spring boot', 'spring framework',
       'sql', 't-sql', 'tcp/ip', 'teamwork', 'telecommunications', 'tfs',
       'typescript', 'uml', 'visual studio', 'web development',
       'web geliştirme', 'web services', 'xml', 'yazılım geliştirme',
       'yönetim', 'company(1th)', 'company(2th)', 'company(3th)',
       'company_location(1th)', 'company_location(2th)',
       'company_location(3th)', 'min_exp_time', 'max_exp_time',
       'company_count_2018', 'company_count_2017', 'company_count_2016']




In [ ]:
scaler = MinMaxScaler()
train_df[features] = scaler.fit_transform(train_df[features])
test_df[features] = scaler.transform(test_df[features])

X_train = train_df[features]
y_train = train_df['moved_after_2019']
svc = SVC()
svc.fit(X_train, y_train)
X_test = test_df[features]
y_pred = svc.predict(X_test)




In [ ]:
df_subm.loc[test_df.index, target] = svc.predict(test_df)

df_subm[target] = df_subm[target].astype(int)
df_subm[target].value_counts()



In [ ]:
df_subm.to_csv('submission.csv')


In [ ]:

X_train = train_df[features].values
y_train = train_df['moved_after_2019'].values
X_test = test_df[features].values

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svc = SVC()
svc.fit(X_train_scaled, y_train)


In [ ]:
df_subm.loc[test_df.index, target] = svc.predict(test_df)

df_subm[target] = df_subm[target].astype(int)
df_subm[target].value_counts()



#gradiandt booster

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


X = train_df.drop(columns=[target])
y =  train_df[target]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:


model = KNeighborsClassifier(n_neighbors=5, weights='uniform')
model.fit(X_train, y_train)
score = model.score(X_test, y_test)
print('KNN model score is %0.4f' %score)

In [ ]:


# Create a DataFrame for the submission
submission = pd.DataFrame(columns=X_test.columns)

# Predict the target variable for the test set
submission[target] = model.predict(X_test)

# Convert the target variable to integer type
submission[target] = submission[target].astype(int)

# Output the predictions to a CSV file
submission.to_csv('submission.csv', index=False)



In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier

scaler = MinMaxScaler()
train_df[features] = scaler.fit_transform(train_df[features])
test_df[features] = scaler.transform(test_df[features])

# Train the model
X_train = train_df[features]
y_train = train_df['moved_after_2019']
gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)






In [ ]:
df_subm.loc[test_df.index, target] = gbc.predict(test_df)

df_subm[target] = df_subm[target].astype(int)
df_subm[target].value_counts()


In [ ]:
df_subm.to_csv('submission.csv')


logistic reggresion

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression

scaler = MinMaxScaler()
train_features = scaler.fit_transform(train_df[features])
train_labels = train_df['moved_after_2019']

# Train logistic regression model
logreg = LogisticRegression()
logreg.fit(train_features, train_labels)

# Make predictions on test data
test_features = scaler.transform(test_df[features])
predictions = logreg.predict(test_features)


In [ ]:
df_subm.loc[test_df.index, target] = logreg.predict(test_df)

df_subm[target] = df_subm[target].astype(int)
df_subm[target].value_counts()


In [ ]:
df_subm.to_csv('submission.csv')


decision tree

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)

# Make predictions on test data
X_test = test_df[features]
y_pred = dtc.predict(X_test)


In [ ]:
df_subm.loc[test_df.index, target] = dtc.predict(test_df)

df_subm[target] = df_subm[target].astype(int)
df_subm[target].value_counts()


In [ ]:
df_subm.to_csv('submission.csv')


KNeighborsClassifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import pandas as pd

In [ ]:
k = 5  # number of neighbors to consider
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train, y_train)
predictions = knn.predict(X_test)

In [ ]:
df_subm.loc[test_df.index, target] = knn.predict(test_df)

df_subm[target] = df_subm[target].astype(int)
df_subm[target].value_counts()


In [ ]:
df_subm.to_csv('submission.csv')


Naive Bayes Classifier

In [ ]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [ ]:
nbc = GaussianNB()
nbc.fit(X_train, y_train)

# Predict on test set
y_pred = nbc.predict(X_test)

In [ ]:
df_subm.loc[test_df.index, target] = nbc.predict(test_df)

df_subm[target] = df_subm[target].astype(int)
df_subm[target].value_counts()


In [ ]:
df_subm.to_csv('submission.csv')


xgboost

In [ ]:
pip install xgboost


In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:

X = train_df.drop(columns=[target], axis=1)
y = train_df[target]


# split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
import xgboost as xgb

# Convert data to DMatrix format
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)

# Set parameters for XGBoost
params = {
    'max_depth': 3,
    'eta': 0.1,
    'objective': 'binary:logistic',
    'eval_metric': 'logloss'
}

# Train the model
num_rounds = 100
xgb_model = xgb.train(params, dtrain, num_rounds)

# Make predictions on test set
y_pred = xgb_model.predict(dtest)

# Convert probabilities to binary predictions
y_pred_binary = [1 if prob >= 0.5 else 0 for prob in y_pred]


In [ ]:
import xgboost as xgb
# One-hot encode categorical columns in test_df
test_df = pd.get_dummies(test_df, columns=['industry', 'location'])

# Ensure that the test data has the same columns as the training data
missing_cols = set(X_train.columns) - set(test_df.columns)
for col in missing_cols:
    test_df[col] = 0
test_df = test_df[X_train.columns]

# Convert testing data to DMatrix format
dtest = xgb.DMatrix(test_df)

# Convert training and testing data to DMatrix format
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(test_df)

# Set parameters for XGBoost
params = {
    'max_depth': 3,
    'eta': 0.1,
    'objective': 'binary:logistic',
    'eval_metric': 'logloss'
}

# Train the model
num_rounds = 100
xgb_model = xgb.train(params, dtrain, num_rounds)

# Make predictions on test set
y_pred = xgb_model.predict(dtest)



In [ ]:

# Train the model
params = {
    'max_depth': 6,
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'eta': 0.3,
    'min_child_weight': 1,
    'subsample': 1,
    'colsample_bytree': 1,
    'seed': 42
}

d_train = xgb.DMatrix(X_train, label=y_train)
d_test = xgb.DMatrix(X_test, label=y_test)

model = xgb.train(params, d_train, 100)

# Make predictions on the test set
y_pred = model.predict(d_test)

# Evaluate the model
y_pred_labels = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, y_pred_labels)

print("Accuracy:", accuracy)


In [ ]:
from xgboost import XGBClassifier

# ***********create XGBoost Classifier object
xgb = XGBClassifier()

# train the model
xgb.fit(X_train, y_train)

# make predictions on test set
y_pred = xgb.predict(X_test)

# evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# make predictions on submission dataset and save to dataframe
df_subm.loc[test_df.index, target] = xgb.predict(test_df)
df_subm[target] = df_subm[target].astype(int)
df_subm[target].value_counts()


In [ ]:
df_subm.to_csv('submission.csv')


In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier()

# train the model
xgb.fit(X_train, y_train)

# make predictions on test set
y_pred = xgb.predict(X_test)

# evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# make predictions on submission dataset and save to dataframe
df_subm.loc[test_df.index, target] = xgb.predict(test_df)
df_subm[target] = df_subm[target].astype(int)
df_subm[target].value_counts()

In [ ]:
df_subm.to_csv('submission.csv')


In [ ]:
df_subm.loc[test_df.index, target] = y_pred.predict(test_df)

df_subm[target] = df_subm[target].astype(int)
df_subm[target].value_counts()


In [ ]:

df_subm = pd.DataFrame({'user_id':  X_test.index, 'moved_after_2019': y_pred})

# save the DataFrame to a CSV file
df_subm.to_csv('submission.csv', index=False)

In [ ]:
clf.fit(X, y)
df_subm.loc[df_test.index, target] = clf.predict(df_test)

df_subm[target] = df_subm[target].astype(int)
df_subm[target].value_counts()

In [ ]:
# Drop rows with missing or invalid values
train_df = train_df.dropna()

# Split data into X and y variables
X, y = train_df.drop(columns=[target]), train_df[target]

# Create classifier and cross-validation objects
clf = RandomForestClassifier(n_estimators=200, class_weight='balanced_subsample', random_state=42)
cv  = StratifiedKFold(shuffle=True, random_state=42)

# Calculate accuracy scores using cross-validation
scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')


In [ ]:
# Calculate accuracy scores using cross-validation
scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')

# Print mean and standard deviation of accuracy scores
print('Accuracy: %0.2f (+/- %0.2f)' % (scores.mean(), scores.std() * 2))


In [ ]:
test_df.reset_index(drop=True, inplace=True)

index = user_id

In [ ]:
clf.fit(X, y)
df_subm.loc[index, target] = clf.predict(test_df)

df_subm[target] = df_subm[target].astype(int)

# Get the counts of 0 and 1 in the target column
print(y.value_counts())

# Get the counts of 0 and 1 in the submission dataframe
print(df_subm[target].value_counts())


In [ ]:
df_subm.to_csv('submission.csv')